In [15]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
import sqlite3

In [8]:
df_1 = pd.read_csv(fr"C:\Users\AMONTORIOP002\Downloads\data\data\test_data_BankIA.csv", encoding="utf-8", sep=";")
df_1.head(3)

,Unnamed: 0,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,...,Country from,Country to,Merchant Type,Device Used,Transaction Type,IP Address,Distance,Previous Transactions,Time Since Last Transaction,ID
0,1609926,2022-10-01 00:40:55,37044,802A148D0,1047,802A16D70,97.44,US Dollar,97.44,US Dollar,...,IT,IT,Grocery,ATM,Wire Transfer,250.244.98.96,487.421221,22.0,60.390602,20221001004055_1855
1,1489357,2022-10-04 21:46:28,3229,80B662790,12803,80E3D8340,31955.39,US Dollar,31955.39,US Dollar,...,ES,ES,Retail,ATM,In-Person Payment,205.239.85.150,154.204181,35.0,0.000000,20221004214628_3499
2,4828860,2022-10-03 13:52:44,70,100428978,140754,80FCD30B0,6145.40,Swiss Franc,6145.40,Swiss Franc,...,FR,FR,Grocery,Mobile,In-Person Payment,122.74.136.105,189.592775,27.0,110.820536,20221003135244_9932


In [ ]:
df = pd.read_csv(fr"C:\Users\AMONTORIOP002\Downloads\data\data\training_data_BankIA.csv", encoding="utf-8", sep=";")
df.head(3)

,Unnamed: 0,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,...,Country from,Country to,Merchant Type,Device Used,Transaction Type,IP Address,Distance,Previous Transactions,Time Since Last Transaction,ID
0,4676453,2022-09-14 01:14:06,322108,8080C2D70,23525,8080C1170,200.63,Euro,200.63,Euro,...,GB,GB,Retail,Desktop,Online Purchase,206.174.206.75,260.700035,84.0,268.631292,20220914011406_3398
1,4765130,2022-09-19 08:04:00,513,80188E150,11474,8021E0E50,64778.64,US Dollar,64778.64,US Dollar,...,ES,ES,Grocery,Desktop,Online Purchase,224.201.232.196,188.847207,162.0,157.559804,20220919080400_7700
2,3718885,2022-09-11 18:32:14,20846,80ABE67D0,20,812F8E950,718.03,Euro,718.03,Euro,...,ES,ES,Grocery,ATM,In-Person Payment,186.220.207.25,379.572327,52.0,194.958558,20220911183214_1471


In [12]:
df_total= pd.concat([df,df_1], axis=0)
df_total

,Unnamed: 0,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,...,Country from,Country to,Merchant Type,Device Used,Transaction Type,IP Address,Distance,Previous Transactions,Time Since Last Transaction,ID
0,4676453,2022-09-14 01:14:06,322108,8080C2D70,23525,8080C1170,200.63,Euro,200.63,Euro,...,GB,GB,Retail,Desktop,Online Purchase,206.174.206.75,260.700035,84.0,268.631292,20220914011406_3398
1,4765130,2022-09-19 08:04:00,513,80188E150,11474,8021E0E50,64778.64,US Dollar,64778.64,US Dollar,...,ES,ES,Grocery,Desktop,Online Purchase,224.201.232.196,188.847207,162.0,157.559804,20220919080400_7700
2,3718885,2022-09-11 18:32:14,20846,80ABE67D0,20,812F8E950,718.03,Euro,718.03,Euro,...,ES,ES,Grocery,ATM,In-Person Payment,186.220.207.25,379.572327,52.0,194.958558,20220911183214_1471
3,4457387,2022-09-26 18:26:39,10,800220380,21611,801665530,2356.36,US Dollar,2356.36,US Dollar,...,JP,JP,Entertainment,Mobile,ATM Withdrawal,76.78.50.104,460.020395,69.0,47.505917,20220926182639_6310
4,2292500,2022-09-14 12:48:10,21550,8066CEEA0,7478,810394A60,390.86,US Dollar,390.86,US Dollar,...,ES,ES,Grocery,Mobile,Wire Transfer,53.16.135.231,376.152108,185.0,26.630861,20220914124810_2120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675849,4704344,2022-10-14 15:22:24,25768,80BA51AA0,3123,80EC18460,3875.60,Euro,3875.60,Euro,...,IT,IT,Entertainment,Mobile,Online Purchase,106.60.152.176,354.483990,80.0,277.306532,20221014152224_6309
1675850,3338134,2022-10-05 05:26:11,13862,80DCCA880,19542,80DCCA8D0,741.99,Euro,741.99,Euro,...,ES,ES,Luxury Goods,Desktop,In-Person Payment,239.132.129.18,220.946094,159.0,291.996757,20221005052611_5534
1675851,1368281,2022-10-06 23:00:48,1688,80187E4F0,227720,8147C9960,358.35,US Dollar,358.35,US Dollar,...,ES,ES,Grocery,Desktop,Online Purchase,189.73.149.198,225.237361,4.0,21.141423,20221006230048_6583
1675852,2231736,2022-10-15 01:17:12,117070,8064A9D40,217824,806F75230,11179.44,Ruble,11179.44,Ruble,...,ES,ES,Retail,Desktop,ATM Withdrawal,229.130.140.82,19.339131,120.0,309.975036,20221015011712_9276


In [ ]:
table_name = "FOUNDATIONS"
sep = ";"
encoding = "utf-8"

conn = sqlite3.connect(fr'db/{table_name}.db')

df = df_total

df.to_sql(table_name, conn, if_exists='replace', index=False)
# Cierra la conexión
conn.close()

In [19]:
pd.read_csv(fr"C:\Users\AMONTORIOP002\Downloads\test1.csv")

,Unnamed: 0,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,...,Country from,Country to,Merchant Type,Device Used,Transaction Type,IP Address,Distance,Previous Transactions,Time Since Last Transaction,ID
0,1609926,2022-10-01 00:40:55,37044,802A148D0,1047,802A16D70,97.44,US Dollar,97.44,US Dollar,...,IT,IT,Grocery,ATM,Wire Transfer,250.244.98.96,487.421221,22.0,60.390602,20221001004055_1855


In [ ]:
import pickle

with open('./model/best_model.pkl', 'rb') as file:
        model = pickle.load(file)

ModuleNotFoundError: No module named 'category_encoders'